# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\elizabeth.xu\AppData\Local\Continuum\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted

pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [4]:
df = df.join(pd.get_dummies(df['prestige']))
print df

     admit    gre   gpa  prestige  1.0  2.0  3.0  4.0
0        0  380.0  3.61       3.0    0    0    1    0
1        1  660.0  3.67       3.0    0    0    1    0
2        1  800.0  4.00       1.0    1    0    0    0
3        1  640.0  3.19       4.0    0    0    0    1
4        0  520.0  2.93       4.0    0    0    0    1
5        1  760.0  3.00       2.0    0    1    0    0
6        1  560.0  2.98       1.0    1    0    0    0
7        0  400.0  3.08       2.0    0    1    0    0
8        1  540.0  3.39       3.0    0    0    1    0
9        0  700.0  3.92       2.0    0    1    0    0
10       0  800.0  4.00       4.0    0    0    0    1
11       0  440.0  3.22       1.0    1    0    0    0
12       1  760.0  4.00       1.0    1    0    0    0
13       0  700.0  3.08       2.0    0    1    0    0
14       1  700.0  4.00       1.0    1    0    0    0
15       0  480.0  3.44       3.0    0    0    1    0
16       0  780.0  3.87       4.0    0    0    0    1
17       0  360.0  2.56     

#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3, since there are a total of 4 options for prestige

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [5]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

In [6]:
df.count()

admit       397
gre         397
gpa         397
prestige    397
1.0         397
2.0         397
3.0         397
4.0         397
dtype: int64

In [7]:
df.sum()

admit          126.00
gre         233380.00
gpa           1346.72
prestige       988.00
1.0             61.00
2.0            148.00
3.0            121.00
4.0             67.00
dtype: float64

In [8]:
df.groupby('admit').sum()

,gre,gpa,prestige,1.0,2.0,3.0,4.0
admit,,,,,,,
0,155440.0,907.08,717.0,28,95,93,55
1,77940.0,439.64,271.0,33,53,28,12


In [9]:
#1: 33:28
#2: 53:95
#3: 28:93
#4: 12:55

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [10]:
#If you attended a #1 prestige college, your odds are 33 to 28

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [11]:
#If you attended a #1 prestige college, your odds are (53+28+12) to (95+93+55) or 93 to 243

#### 3.3 Calculate the odds ratio

In [12]:
#1: 33:28
#2: 53:95
#3: 28:93
#4: 12:55

#### 3.4 Write this finding in a sentenance: 

#Answer: The Odds ratio descreases as one's college prestige decreases.

#### 3.5 Print the cross tab for prestige_4

In [13]:
pd.crosstab(df['admit'], df[4.0], rownames=['admit'])

4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [14]:
#What is OR, if it's pdd ratio, then it is 12 to 55.

#### 3.7 Write this finding in a sentence

#Answer: Ignoring GPA and GMAT, the odds of getting accepted from a prestige 4 school is 12 to 55.

## Part 4. Analysis

In [15]:
# create a clean data frame for the regression
df_clean = df.dropna() 
print df_clean.head()

   admit    gre   gpa  prestige  1.0  2.0  3.0  4.0
0      0  380.0  3.61       3.0    0    0    1    0
1      1  660.0  3.67       3.0    0    0    1    0
2      1  800.0  4.00       1.0    1    0    0    0
3      1  640.0  3.19       4.0    0    0    0    1
4      0  520.0  2.93       4.0    0    0    0    1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [23]:
# manually add the intercept
X = df_clean[['gre', 'gpa', 1.0, 2.0, 3.0]]
X = sm.add_constant(X)
y = df_clean['admit']

#### 4.1 Set the covariates to a variable called train_cols

In [25]:
train_cols = df_clean.columns[1:]
print X

     const    gre   gpa  1.0  2.0  3.0
0      1.0  380.0  3.61    0    0    1
1      1.0  660.0  3.67    0    0    1
2      1.0  800.0  4.00    1    0    0
3      1.0  640.0  3.19    0    0    0
4      1.0  520.0  2.93    0    0    0
5      1.0  760.0  3.00    0    1    0
6      1.0  560.0  2.98    1    0    0
7      1.0  400.0  3.08    0    1    0
8      1.0  540.0  3.39    0    0    1
9      1.0  700.0  3.92    0    1    0
10     1.0  800.0  4.00    0    0    0
11     1.0  440.0  3.22    1    0    0
12     1.0  760.0  4.00    1    0    0
13     1.0  700.0  3.08    0    1    0
14     1.0  700.0  4.00    1    0    0
15     1.0  480.0  3.44    0    0    1
16     1.0  780.0  3.87    0    0    0
17     1.0  360.0  2.56    0    0    1
18     1.0  800.0  3.75    0    1    0
19     1.0  540.0  3.81    1    0    0
20     1.0  500.0  3.17    0    0    1
21     1.0  660.0  3.63    0    1    0
22     1.0  600.0  2.82    0    0    0
23     1.0  680.0  3.19    0    0    0
24     1.0  760.0  3.35  

#### 4.2 Fit the model

In [26]:
logit = sm.Logit(y, X)

#### 4.3 Print the summary results

In [27]:
result = logit.fit()
print result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 04 Oct 2017   Pseudo R-squ.:                 0.08166
Time:                        18:53:51   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.4303      1.140     -4.764      0.000      -7.664      -3.196
gre            0.0022      0.

In [30]:
print result.predict(X)

0      0.173771
1      0.290859
2      0.734040
3      0.178814
4      0.119915
5      0.369819
6      0.422597
7      0.219382
8      0.201710
9      0.512721
10     0.368608
11     0.403407
12     0.716361
13     0.353486
14     0.688556
15     0.186976
16     0.335397
17     0.081525
18     0.535004
19     0.572084
20     0.163033
21     0.434417
22     0.129940
23     0.192218
24     0.435309
25     0.679232
26     0.577364
27     0.203920
28     0.421366
29     0.460268
         ...   
370    0.396289
371    0.315030
372    0.381699
373    0.531190
374    0.408833
375    0.187236
376    0.412755
377    0.582995
378    0.203555
379    0.221863
380    0.460202
381    0.345435
382    0.348968
383    0.669216
384    0.189956
385    0.356285
386    0.523201
387    0.342597
388    0.339233
389    0.400895
390    0.400042
391    0.482753
392    0.222643
393    0.435593
394    0.252465
395    0.483948
396    0.167418
397    0.184392
398    0.460202
399    0.298832
Length: 397, dtype: floa

In [31]:
print result.conf_int()

              0         1
const -7.664377 -3.196152
gre    0.000074  0.004362
gpa    0.127619  1.431056
1.0    0.735197  2.371624
2.0    0.153432  1.593115
3.0   -0.554669  0.984135


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [32]:
print np.exp(result.params)

const    0.004382
gre      1.002221
gpa      2.180027
1.0      4.727566
2.0      2.394738
3.0      1.239531
dtype: float64


In [33]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

           2.5%      97.5%        OR
const  0.000469   0.040919  0.004382
gre    1.000074   1.004372  1.002221
gpa    1.136120   4.183113  2.180027
1.0    2.085893  10.714780  4.727566
2.0    1.165829   4.919050  2.394738
3.0    0.574262   2.675498  1.239531


#### 4.5 Interpret the OR of Prestige_2

#### Answer:The OR of Prestige_2 is 2.394738. The shows an increase of 2.39x of admit for presitge 2 vs. prestiage 4.

#### 4.6 Interpret the OR of GPA

Answer: The OR of GPA is 2.180027. This OR tells you a unit increase in GPA will have an increase of 200% of admit.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [34]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [35]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(df_clean['gre'].min(), df_clean['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(df_clean['gpa'].min(), df_clean['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1.0, 2.0, 3.0, 4.0], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [36]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])


C:\Users\elizabeth.xu\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


#### 5.2 Make predictions on the enumerated dataset

In [38]:
combos['admit_pred'] = result.predict(X)

In [42]:
print combos.tail()

       gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
395  800.0  3.806667       4.0        1.0           0           0           1   
396  800.0  4.000000       1.0        1.0           0           0           0   
397  800.0  4.000000       2.0        1.0           1           0           0   
398  800.0  4.000000       3.0        1.0           0           1           0   
399  800.0  4.000000       4.0        1.0           0           0           1   

     admit_pred  
395    0.483948  
396    0.167418  
397    0.184392  
398    0.460202  
399    0.298832  


#### 5.3 Interpret findings for the last 4 observations

In [43]:
#Answer: The number indicates the probability of the indiviudal being admitted.


In [44]:
## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

SyntaxError: invalid syntax (<ipython-input-44-b26a8f58e3d8>, line 3)